In [1]:
# Imports for making dataframe
import pandas as pd
import numpy as np

In [2]:
# Import for crawling Goodreads
import requests

In [3]:
# Imports for XML
from xml.etree import ElementTree

In [4]:
# Import for finding texts in HTML
from bs4 import BeautifulSoup

In [5]:
# Imports for seeing full text reviews
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
# Goodreads API Developer key
key = 'mTBsGSOvfBaN0OF25U5g'
secret = 'AkIOvUnGYksPPicdwWfQVStbr61O0YzzrVu1ZUmpGA'

In [7]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Goodreads login
email = ''
password = ''

In [8]:
## NOTE: FILL IN BEFORE RUNNING CODE
# Load Amazon data to find reviews of same books from Goodreads
amazon = pd.read_csv('amazonData.csv')
amazon

,overall,reviewTime,asin,reviewText
0,5.0,"03 30, 2005",0001713353,"The King, the Mice and the Cheese by Nancy Gur..."
1,5.0,"12 27, 2012",0001713353,This was one of my favorites when I was a smal...
2,5.0,"08 15, 2011",0001713353,"So, you think you have problems? Things could ..."
3,5.0,"11 24, 2008",0001713353,My mom bought this for my oldest son. He and ...
4,3.0,"09 18, 2006",0001713353,41 years later:\n\nThe cheese is government ch...
...,...,...,...,...
115439,5.0,"11 27, 1999",0030554179,"""All animals are equal, but some are more equa..."
115440,5.0,"09 9, 1999",0030554179,It's a story about animals who take over a far...
115441,5.0,"07 27, 1999",0030554179,We should not be content to reduce this book t...
115442,5.0,"06 5, 1999",0030554179,George Orwell is the pseudonym of Eric A. Blai...


In [9]:
# Unique set of books to crawl from Goodreads
books = set(amazon.asin)
print(len(books))

3007


In [10]:
### NOTE: COMMENT OUT FOR FULL SET
## TEST SUBSET
test = set()
for i,val in enumerate(books):
    if i == 3:
        break
    test.add(val)
books = test
books

{'0001712845', '000231312X', '0007121733'}

In [11]:
# Create empty dataframe to store Goodreads reviews & ratings
column_names = ['overall', 'reviewTime', 'asin', 'reviewText']
df = pd.DataFrame(columns = column_names)
print(df)

Empty DataFrame
Columns: [overall, reviewTime, asin, reviewText]
Index: []


In [18]:
# Log in Goodreads
driver = webdriver.Chrome()
driver.get ('https://www.goodreads.com/user/sign_in?source=home')
driver.find_element_by_name('user[email]').send_keys(email)
driver.find_element_by_name('user[password]').send_keys(password)
driver.find_element_by_name('next').click()                            

In [22]:
# Get most popular reviews for each book (10 if there are more than 10 reviews, or all reviews if less)
for book in books:
    
    # Try to look up the book in Goodreads. If a book isn't found it will not be added to the dataset -> DATA CLEANING LATER
    try:
        #print('TRYING BOOK:', book)
        
        # Search up book on Goodreads
        res = requests.get('https://www.goodreads.com/book/isbn', params={'format': 'xml', 'key': key, 'isbn': book})
        
        # Get XML from API
        root = ElementTree.fromstring(res.text)    
        
        # Find iframe from XML
        iframe = root.find('book').find('reviews_widget').text
        
        # Parse HTML of XML to find link to reviews
        soup = BeautifulSoup(iframe, 'html.parser')
        url = soup.find('iframe')['src']
        
        # Open link to reviews using Selenium
        driver.get(str(url))

        # PAGE 1: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
                            'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        # PAGE 2: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        driver.find_element_by_link_text('2').click()
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
                            'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()           
            driver.switch_to.window(driver.window_handles[0])
            
        # PAGE 3: Expand each review to save full text & metadata
        # If a book doesn't have reviews, it will not be added to the dataset -> DATA CLEANING LATER
        driver.find_element_by_link_text('3').click()
        links = driver.find_elements_by_link_text('...more') 

        for link in links:
            # Open review in new tab
            ActionChains(driver).move_to_element(link).perform()
            link.click()
            
            # Switch to new tab
            driver.switch_to.window(driver.window_handles[1])
            s = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Use BeautifulSoup to check & save reviewTime
            time = s.find('span', itemprop='datePublished')
            
            # Use BeautifulSoup to save rating
            rating = s.find('meta', itemprop='ratingValue')
            
            # Use BeautifulSoup to save text review
            review = s.find('div', itemprop='reviewBody')
            
            # Add review to dataframe
            df = df.append({'overall':rating.get('content'), 'reviewTime':time.text.strip(), 'asin':book, 
                            'reviewText':review.text.strip()}, ignore_index=True)
                
            driver.close()           
            driver.switch_to.window(driver.window_handles[0])
            
    except:
        pass
        #print('   NOT FOUND')

TRYING BOOK: 0007121733
https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=0007121733&links=660&min_rating=&review_back=fff&stars=000&text=000
TRYING BOOK: 000231312X
https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=000231312X&links=660&min_rating=&review_back=fff&stars=000&text=000
TRYING BOOK: 0001712845
https://www.goodreads.com/api/reviews_widget_iframe?did=DEVELOPER_ID&format=html&isbn=0001712845&links=660&min_rating=&review_back=fff&stars=000&text=000


In [23]:
df

,overall,reviewTime,asin,reviewText
0,5,"May 06, 2018",0007121733,(1) recently I went on vacation with a guy who...
1,5,"Jun 12, 2012",0007121733,The recent popularity of THE CHANGEOVER can pr...
2,4,"Jan 18, 2010",0007121733,I wish I had read this book when I was 16 and ...
3,5,"Aug 29, 2009",0007121733,Margaret Mahy's The Tricksters was part of my ...
4,5,"May 07, 2016",0007121733,It's hard not to give Mahy books all the stars...
...,...,...,...,...
85,5,"Aug 07, 2007",0001712845,I always picked this book for my mom to read t...
86,5,"Oct 08, 2011",0001712845,This is probobly the best Berenstain book out ...
87,4,"Jun 06, 2008",0001712845,One of my children's favorites this book is bo...
88,5,"Sep 04, 2011",0001712845,My favorite book to read to my kids and now my...


In [25]:
## NOTE: CHANGE FILE NAME BEFORE RUNNING CODE
df.to_csv('grData.csv',index=False)